<a rel="license" href="http://creativecommons.org/licenses/by-nc-nd/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-nd/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc-nd/4.0/">Creative Commons Attribution-NonCommercial-NoDerivatives 4.0 International License</a>.

#![Spark Logo](http://spark-mooc.github.io/web-assets/images/ta_Spark-logo-small.png) + ![Python Logo](http://spark-mooc.github.io/web-assets/images/python-logo-master-v3-TM-flattened_small.png)
# **Spark Tutorial: Learning Apache Spark**

This tutorial will teach you how to use [Apache Spark](http://spark.apache.org/), a framework for large-scale data processing, within a notebook. Many traditional frameworks were designed to be run on a single computer.  However, many datasets today are too large to be stored on a single computer, and even when a dataset can be stored on one computer (such as the datasets in this tutorial), the dataset can often be processed much more quickly using multiple computers.

Spark has efficient implementations of a number of transformations and actions that can be composed together to perform data processing and analysis.  Spark excels at distributing these operations across a cluster while abstracting away many of the underlying implementation details.  Spark has been designed with a focus on scalability and efficiency.  With Spark you can begin developing your solution on your laptop, using a small dataset, and then use that same code to process terabytes or even petabytes across a distributed cluster.

**During this tutorial we will cover:**

* *Part 1:* Basic notebook usage and [Python](https://docs.python.org/2/) integration
* *Part 2:* An introduction to using [Apache Spark](https://spark.apache.org/) with the [PySpark SQL API](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark-sql-module) running in a notebook
* *Part 3:* Using DataFrames and chaining together transformations and actions
* *Part 4*: Python Lambda functions and User Defined Functions
* *Part 5:* Additional DataFrame actions
* *Part 6:* Additional DataFrame transformations
* *Part 7:* Caching DataFrames and storage options
* *Part 8:* Debugging Spark applications and lazy evaluation

The following transformations will be covered:
* `select()`, `filter()`, `distinct()`, `dropDuplicates()`, `orderBy()`, `groupBy()`

The following actions will be covered:
* `first()`, `take()`, `count()`, `collect()`, `show()`

Also covered:
* `cache()`, `unpersist()`

Note that, for reference, you can look up the details of these methods in the [Spark's PySpark SQL API](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark-sql-module)

## **Part 1: Basic notebook usage and [Python](https://docs.python.org/2/) integration **

### (1a) Notebook usage

A notebook is comprised of a linear sequence of cells.  These cells can contain either markdown or code, but we won't mix both in one cell.  When a markdown cell is executed it renders formatted text, images, and links just like HTML in a normal webpage.  The text you are reading right now is part of a markdown cell.  Python code cells allow you to execute arbitrary Python commands just like in any Python shell. Place your cursor inside the cell below, and press "Shift" + "Enter" to execute the code and advance to the next cell.  You can also press "Ctrl" + "Enter" to execute the code and remain in the cell.  These commands work the same in both markdown and code cells.

In [5]:
# This is a Python cell. You can run normal Python code here...
print 'The sum of 1 and 1 is {0}'.format(1+1)

The sum of 1 and 1 is 2

In [6]:
# Here is another Python cell, this time with a variable (x) declaration and an if statement:
x = 42
if x > 40:
    print 'The sum of 1 and 2 is {0}'.format(1+2)

The sum of 1 and 2 is 3

### (1b) Notebook state

As you work through a notebook it is important that you run all of the code cells.  The notebook is stateful, which means that variables and their values are retained until the notebook is detached (in Databricks) or the kernel is restarted (in Jupyter notebooks).  If you do not run all of the code cells as you proceed through the notebook, your variables will not be properly initialized and later code might fail.  You will also need to rerun any cells that you have modified in order for the changes to be available to other cells.

In [8]:
# This cell relies on x being defined already.
# If we didn't run the cells from part (1a) this code would fail.
print x * 2

84

### (1c) Library imports

We can import standard Python libraries ([modules](https://docs.python.org/2/tutorial/modules.html)) the usual way.  An `import` statement will import the specified module.  In this tutorial and future labs, we will provide any imports that are necessary.

In [10]:
# Import the regular expression library
import re
m = re.search('(?<=abc)def', 'abcdef')
m.group(0)

Out[ 4 ]: 'def'

In [11]:
# Import the datetime library
import datetime
print 'This was last run on: {0}'.format(datetime.datetime.now())

This was last run on: 2016-07-15 16:46:31.414638

##  **Part 2: An introduction to using [Apache Spark](https://spark.apache.org/) with the [PySpark SQL API](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark-sql-module) running in a notebook**

### Spark Context

In Spark, communication occurs between a driver and executors.  The driver has Spark jobs that it needs to run and these jobs are split into tasks that are submitted to the executors for completion.  The results from these tasks are delivered back to the driver.

In part 1, we saw that normal Python code can be executed via cells. When using Databricks this code gets executed in the Spark driver's Java Virtual Machine (JVM) and not in an executor's JVM, and when using an Jupyter notebook it is executed within the kernel associated with the notebook. Since no Spark functionality is actually being used, no tasks are launched on the executors.

In order to use Spark and its DataFrame API we will need to use a `SQLContext`.  When running Spark, you start a new Spark application by creating a [SparkContext](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.SparkContext). You can then create a [SQLContext](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.SQLContext) from the `SparkContext`. When the `SparkContext` is created, it asks the master for some cores to use to do work.  The master sets these cores aside just for you; they won't be used for other applications. When using Databricks, both a `SparkContext` and a `SQLContext` are created for you automatically. `sc` is your `SparkContext`, and `sqlContext` is your `SQLContext`.

### (2a) Example Cluster
The diagram shows an example cluster, where the slots allocated for an application are outlined in purple. (Note: We're using the term _slots_ here to indicate threads available to perform parallel work for Spark.
Spark documentation often refers to these threads as _cores_, which is a confusing term, as the number of slots available on a particular machine does not necessarily have any relationship to the number of physical CPU
cores on that machine.)

<img src="http://spark-mooc.github.io/web-assets/images/cs105x/diagram-2a.png" style="height: 800px;float: right"/>

You can view the details of your Spark application in the Spark web UI.  The web UI is accessible in Databricks by going to "Clusters" and then clicking on the "Spark UI" link for your cluster.  In the web UI, under the "Jobs" tab, you can see a list of jobs that have been scheduled or run.  It's likely there isn't any thing interesting here yet because we haven't run any jobs, but we'll return to this page later.

At a high level, every Spark application consists of a driver program that launches various parallel operations on executor Java Virtual Machines (JVMs) running either in a cluster or locally on the same machine. In Databricks, "Databricks Shell" is the driver program.  When running locally, `pyspark` is the driver program. In all cases, this driver program contains the main loop for the program and creates distributed datasets on the cluster, then applies operations (transformations & actions) to those datasets.
Driver programs access Spark through a SparkContext object, which represents a connection to a computing cluster. A Spark SQL context object (`sqlContext`) is the main entry point for Spark DataFrame and SQL functionality. A `SQLContext` can be used to create DataFrames, which allows you to direct the operations on your data.

Try printing out `sqlContext` to see its type.

In [15]:
# Display the type of the Spark sqlContext
type(sqlContext)

Out[ 6 ]: pyspark.sql.context.HiveContext

Note that the type is `HiveContext`. This means we're working with a version of Spark that has Hive support. Compiling Spark with Hive support is a good idea, even if you don't have a Hive metastore. As the
[Spark Programming Guide](http://spark.apache.org/docs/latest/sql-programming-guide.html#starting-point-sqlcontext) states, a `HiveContext` "provides a superset of the functionality provided by the basic `SQLContext`. Additional features include the ability to write queries using the more complete HiveQL parser, access to Hive UDFs [user-defined functions], and the ability to read data from Hive tables. To use a `HiveContext`, you do not need to have an existing Hive setup, and all of the data sources available to a `SQLContext` are still available."

### (2b) SparkContext attributes

You can use Python's [dir()](https://docs.python.org/2/library/functions.html?highlight=dir#dir) function to get a list of all the attributes (including methods) accessible through the `sqlContext` object.

In [18]:
# List sqlContext's attributes
dir(sqlContext)

Out[ 7 ]: 
['__class__',
 '__delattr__',
 '__dict__',
 '__doc__',
 '__format__',
 '__getattribute__',
 '__hash__',
 '__init__',
 '__module__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_createFromLocal',
 '_createFromRDD',
 '_get_hive_ctx',
 '_inferSchema',
 '_inferSchemaFromList',
 '_instantiatedContext',
 '_jsc',
 '_jvm',
 '_sc',
 '_scala_HiveContext',
 '_scala_SQLContext',
 '_ssql_ctx',
 'applySchema',
 'cacheTable',
 'clearCache',
 'createDataFrame',
 'createExternalTable',
 'dropTempTable',
 'getConf',
 'getOrCreate',
 'inferSchema',
 'jsonFile',
 'jsonRDD',
 'load',
 'newSession',
 'parquetFile',
 'range',
 'read',
 'refreshTable',
 'registerDataFrameAsTable',
 'registerFunction',
 'setConf',
 'sql',
 'table',
 'tableNames',
 'tables',
 'udf',
 'uncacheTable']

### (2c) Getting help

Alternatively, you can use Python's [help()](https://docs.python.org/2/library/functions.html?highlight=help#help) function to get an easier to read list of all the attributes, including examples, that the `sqlContext` object has.

In [20]:
# Use help to obtain more detailed information
help(sqlContext)

Help on HiveContext in module pyspark.sql.context object:

class HiveContext(SQLContext)
 A variant of Spark SQL that integrates with data stored in Hive.
 
 Configuration for Hive is read from ``hive-site.xml`` on the classpath.
 It supports running both SQL and HiveQL commands.
 
 :param sparkContext: The SparkContext to wrap.
 :param hiveContext: An optional JVM Scala HiveContext. If set, we do not instantiate a new
 :class:`HiveContext` in the JVM, instead we make all calls to this object.
 
 Method resolution order:
 HiveContext
 SQLContext
 __builtin__.object
 
 Methods defined here:
 
 __init__(self, sparkContext, hiveContext=None)
 
 refreshTable(self, tableName)
 Invalidate and refresh all the cached the metadata of the given
 table. For performance reasons, Spark SQL or the external data source
 library it uses might cache certain metadata about a table, such as the
 location of blocks. When those change outside of Spark SQL, users should
 call this function to invalidate the cache.
 
 ----------------------------------------------------------------------
 Methods inherited from SQLContext:
 
 applySchema(self, rdd, schema)
 .. note:: Deprecated in 1.3, use :func:`createDataFrame` instead.
 
 cacheTable(self, tableName)
 Caches the specified table in-memory.
 
 .. versionadded:: 1.0
 
 clearCache(self)
 Removes all cached tables from the in-memory cache.
 
 .. versionadded:: 1.3
 
 createDataFrame(self, data, schema=None, samplingRatio=None)
 Creates a :class:`DataFrame` from an :class:`RDD` of :class:`tuple`/:class:`list`,
 list or :class:`pandas.DataFrame`.
 
 When ``schema`` is a list of column names, the type of each column
 will be inferred from ``data``.
 
 When ``schema`` is ``None``, it will try to infer the schema (column names and types)
 from ``data``, which should be an RDD of :class:`Row`,
 or :class:`namedtuple`, or :class:`dict`.
 
 If schema inference is needed, ``samplingRatio`` is used to determined the ratio of
 rows used for schema inference. The first row will be used if ``samplingRatio`` is ``None``.
 
 :param data: an RDD of :class:`Row`/:class:`tuple`/:class:`list`/:class:`dict`,
 :class:`list`, or :class:`pandas.DataFrame`.
 :param schema: a :class:`StructType` or list of column names. default None.
 :param samplingRatio: the sample ratio of rows used for inferring
 :return: :class:`DataFrame`
 
 >>> l = [('Alice', 1)]
 >>> sqlContext.createDataFrame(l).collect()
 [Row(_1=u'Alice', _2=1)]
 >>> sqlContext.createDataFrame(l, ['name', 'age']).collect()
 [Row(name=u'Alice', age=1)]
 
 >>> d = [{'name': 'Alice', 'age': 1}]
 >>> sqlContext.createDataFrame(d).collect()
 [Row(age=1, name=u'Alice')]
 
 >>> rdd = sc.parallelize(l)
 >>> sqlContext.createDataFrame(rdd).collect()
 [Row(_1=u'Alice', _2=1)]
 >>> df = sqlContext.createDataFrame(rdd, ['name', 'age'])
 >>> df.collect()
 [Row(name=u'Alice', age=1)]
 
 >>> from pyspark.sql import Row
 >>> Person = Row('name', 'age')
 >>> person = rdd.map(lambda r: Person(*r))
 >>> df2 = sqlContext.createDataFrame(person)
 >>> df2.collect()
 [Row(name=u'Alice', age=1)]
 
 >>> from pyspark.sql.types import *
 >>> schema = StructType([
 ... StructField("name", StringType(), True),
 ... StructField("age", IntegerType(), True)])
 >>> df3 = sqlContext.createDataFrame(rdd, schema)
 >>> df3.collect()
 [Row(name=u'Alice', age=1)]
 
 >>> sqlContext.createDataFrame(df.toPandas()).collect() # doctest: +SKIP
 [Row(name=u'Alice', age=1)]
 >>> sqlContext.createDataFrame(pandas.DataFrame([[1, 2]])).collect() # doctest: +SKIP
 [Row(0=1, 1=2)]
 
 .. versionadded:: 1.3
 
 createExternalTable(self, tableName, path=None, source=None, schema=None, **options)
 Creates an external table based on the dataset in a data source.
 
 It returns the DataFrame associated with the external table.
 
 The data source is specified by the ``source`` and a set of ``options``.
 If ``source`` is not specified, the default data source configured by
 ``spark.sql.sources.default`` will be used.
 
 Option

Outside of `pyspark` or a notebook, `SQLContext` is created from the lower-level `SparkContext`, which is usually used to create Resilient Distributed Datasets (RDDs). An RDD is the way Spark actually represents data internally; DataFrames are actually implemented in terms of RDDs.

While you can interact directly with RDDs, DataFrames are preferred. They're generally faster, and they perform the same no matter what language (Python, R, Scala or Java) you use with Spark.

In this course, we'll be using DataFrames, so we won't be interacting directly with the Spark Context object very much. However, it's worth knowing that inside `pyspark` or a notebook, you already have an existing `SparkContext` in the `sc` variable. One simple thing we can do with `sc` is check the version of Spark we're using:

In [22]:
# After reading the help we've decided we want to use sc.version to see what version of Spark we are running
sc.version

Out[ 9 ]: u'1.6.1'

In [23]:
# Help can be used on any Python object
help(map)

Help on built-in function map in module __builtin__:

map(...)
 map(function, sequence[, sequence, ...]) -> list
 
 Return a list of the results of applying the function to the items of
 the argument sequence(s). If more than one sequence is given, the
 function is called with an argument list consisting of the corresponding
 item of each sequence, substituting None for missing values when not all
 sequences have the same length. If the function is None, return a list of
 the items of the sequence (or a list of tuples if more than one sequence).

## **Part 3: Using DataFrames and chaining together transformations and actions**

### Working with your first DataFrames

In Spark, we first create a base [DataFrame](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame). We can then apply one or more transformations to that base DataFrame. *A DataFrame is immutable, so once it is created, it cannot be changed.* As a result, each transformation creates a new DataFrame. Finally, we can apply one or more actions to the DataFrames.

> Note that Spark uses lazy evaluation, so transformations are not actually executed until an action occurs.

We will perform several exercises to obtain a better understanding of DataFrames:
* Create a Python collection of 10,000 integers
* Create a Spark DataFrame from that collection
* Subtract one from each value using `map`
* Perform action `collect` to view results
* Perform action `count` to view counts
* Apply transformation `filter` and view results with `collect`
* Learn about lambda functions
* Explore how lazy evaluation works and the debugging challenges that it introduces

A DataFrame consists of a series of `Row` objects; each `Row` object has a set of named columns. You can think of a DataFrame as modeling a table, though the data source being processed does not have to be a table.

More formally, a DataFrame must have a _schema_, which means it must consist of columns, each of which has a _name_ and a _type_. Some data sources have schemas built into them. Examples include RDBMS databases, Parquet files, and NoSQL databases like Cassandra. Other data sources don't have computer-readable schemas, but you can often apply a schema programmatically.

### (3a) Create a Python collection of 10,000 people

We will use a third-party Python testing library called [fake-factory](https://pypi.python.org/pypi/fake-factory/0.5.3) to create a collection of fake person records.

In [27]:
from faker import Factory
fake = Factory.create()
fake.seed(4321)

We're going to use this factory to create a collection of randomly generated people records. In the next section, we'll turn that collection into a DataFrame. We'll use the Spark `Row` class,
because that will help us define the Spark DataFrame schema. There are other ways to define schemas, though; see
the Spark Programming Guide's discussion of [schema inference](http://spark.apache.org/docs/latest/sql-programming-guide.html#inferring-the-schema-using-reflection) for more information. (For instance,
we could also use a Python `namedtuple`.)

In [29]:
# Each entry consists of last_name, first_name, ssn, job, and age (at least 1)
from pyspark.sql import Row
def fake_entry():
  name = fake.name().split()
  return (name[1], name[0], fake.ssn(), fake.job(), abs(2016 - fake.date_time().year) + 1)

In [30]:
# Create a helper function to call a function repeatedly
def repeat(times, func, *args, **kwargs):
    for _ in xrange(times):
        yield func(*args, **kwargs)

In [31]:
data = list(repeat(10000, fake_entry))

`data` is just a normal Python list, containing Python tuples objects. Let's look at the first item in the list:

In [33]:
data[0]

Out[ 15 ]: (u'Harvey', u'Tracey', u'160-37-9051', 'Agricultural engineer', 39)

We can check the size of the list using the Python `len()` function.

In [35]:
len(data)

Out[ 16 ]: 10000

### (3b) Distributed data and using a collection to create a DataFrame

In Spark, datasets are represented as a list of entries, where the list is broken up into many different partitions that are each stored on a different machine.  Each partition holds a unique subset of the entries in the list.  Spark calls datasets that it stores "Resilient Distributed Datasets" (RDDs). Even DataFrames are ultimately represented as RDDs, with additional meta-data.

<img src="http://spark-mooc.github.io/web-assets/images/cs105x/diagram-3b.png" style="width: 900px; float: right; margin: 5px"/>

One of the defining features of Spark, compared to other data analytics frameworks (e.g., Hadoop), is that it stores data in memory rather than on disk.  This allows Spark applications to run much more quickly, because they are not slowed down by needing to read data from disk.
The figure to the right illustrates how Spark breaks a list of data entries into partitions that are each stored in memory on a worker.


To create the DataFrame, we'll use `sqlContext.createDataFrame()`, and we'll pass our array of data in as an argument to that function. Spark will create a new set of input data based on data that is passed in.  A DataFrame requires a _schema_, which is a list of columns, where each column has a name and a type. Our list of data has elements with types (mostly strings, but one integer). We'll supply the rest of the schema and the column names as the second argument to `createDataFrame()`.

Let's view the help for `createDataFrame()`.

In [38]:
help(sqlContext.createDataFrame)

Help on method createDataFrame in module pyspark.sql.context:

createDataFrame(self, data, schema=None, samplingRatio=None) method of pyspark.sql.context.HiveContext instance
 Creates a :class:`DataFrame` from an :class:`RDD` of :class:`tuple`/:class:`list`,
 list or :class:`pandas.DataFrame`.
 
 When ``schema`` is a list of column names, the type of each column
 will be inferred from ``data``.
 
 When ``schema`` is ``None``, it will try to infer the schema (column names and types)
 from ``data``, which should be an RDD of :class:`Row`,
 or :class:`namedtuple`, or :class:`dict`.
 
 If schema inference is needed, ``samplingRatio`` is used to determined the ratio of
 rows used for schema inference. The first row will be used if ``samplingRatio`` is ``None``.
 
 :param data: an RDD of :class:`Row`/:class:`tuple`/:class:`list`/:class:`dict`,
 :class:`list`, or :class:`pandas.DataFrame`.
 :param schema: a :class:`StructType` or list of column names. default None.
 :param samplingRatio: the sample ratio of rows used for inferring
 :return: :class:`DataFrame`
 
 >>> l = [('Alice', 1)]
 >>> sqlContext.createDataFrame(l).collect()
 [Row(_1=u'Alice', _2=1)]
 >>> sqlContext.createDataFrame(l, ['name', 'age']).collect()
 [Row(name=u'Alice', age=1)]
 
 >>> d = [{'name': 'Alice', 'age': 1}]
 >>> sqlContext.createDataFrame(d).collect()
 [Row(age=1, name=u'Alice')]
 
 >>> rdd = sc.parallelize(l)
 >>> sqlContext.createDataFrame(rdd).collect()
 [Row(_1=u'Alice', _2=1)]
 >>> df = sqlContext.createDataFrame(rdd, ['name', 'age'])
 >>> df.collect()
 [Row(name=u'Alice', age=1)]
 
 >>> from pyspark.sql import Row
 >>> Person = Row('name', 'age')
 >>> person = rdd.map(lambda r: Person(*r))
 >>> df2 = sqlContext.createDataFrame(person)
 >>> df2.collect()
 [Row(name=u'Alice', age=1)]
 
 >>> from pyspark.sql.types import *
 >>> schema = StructType([
 ... StructField("name", StringType(), True),
 ... StructField("age", IntegerType(), True)])
 >>> df3 = sqlContext.createDataFrame(rdd, schema)
 >>> df3.collect()
 [Row(name=u'Alice', age=1)]
 
 >>> sqlContext.createDataFrame(df.toPandas()).collect() # doctest: +SKIP
 [Row(name=u'Alice', age=1)]
 >>> sqlContext.createDataFrame(pandas.DataFrame([[1, 2]])).collect() # doctest: +SKIP
 [Row(0=1, 1=2)]
 
 .. versionadded:: 1.3

In [39]:
dataDF = sqlContext.createDataFrame(data, ('last_name', 'first_name', 'ssn', 'occupation', 'age'))

Let's see what type `sqlContext.createDataFrame()` returned.

In [41]:
print 'type of dataDF: {0}'.format(type(dataDF))

type of dataDF: <class 'pyspark.sql.dataframe.DataFrame'>

Let's take a look at the DataFrame's schema and some of its rows.

In [43]:
dataDF.printSchema()

root
-- last_name: string (nullable = true)
-- first_name: string (nullable = true)
-- ssn: string (nullable = true)
-- occupation: string (nullable = true)
-- age: long (nullable = true)

We can register the newly created DataFrame as a named table, using the `registerDataFrameAsTable()` method.

In [45]:
sqlContext.registerDataFrameAsTable(dataDF, 'dataframe')

What methods can we call on this DataFrame?

In [47]:
help(dataDF)

Help on DataFrame in module pyspark.sql.dataframe object:

class DataFrame(__builtin__.object)
 A distributed collection of data grouped into named columns.
 
 A :class:`DataFrame` is equivalent to a relational table in Spark SQL,
 and can be created using various functions in :class:`SQLContext`::
 
 people = sqlContext.read.parquet("...")
 
 Once created, it can be manipulated using the various domain-specific-language
 (DSL) functions defined in: :class:`DataFrame`, :class:`Column`.
 
 To select a column from the data frame, use the apply method::
 
 ageCol = people.age
 
 A more concrete example::
 
 # To create DataFrame using SQLContext
 people = sqlContext.read.parquet("...")
 department = sqlContext.read.parquet("...")
 
 people.filter(people.age > 30).join(department, people.deptId == department.id)) .groupBy(department.name, "gender").agg({"salary": "avg", "age": "max"})
 
 .. note:: Experimental
 
 .. versionadded:: 1.3
 
 Methods defined here:
 
 __getattr__(self, name)
 Returns the :class:`Column` denoted by ``name``.
 
 >>> df.select(df.age).collect()
 [Row(age=2), Row(age=5)]
 
 .. versionadded:: 1.3
 
 __getitem__(self, item)
 Returns the column as a :class:`Column`.
 
 >>> df.select(df['age']).collect()
 [Row(age=2), Row(age=5)]
 >>> df[ ["name", "age"]].collect()
 [Row(name=u'Alice', age=2), Row(name=u'Bob', age=5)]
 >>> df[ df.age > 3 ].collect()
 [Row(age=5, name=u'Bob')]
 >>> df[df[0] > 3].collect()
 [Row(age=5, name=u'Bob')]
 
 .. versionadded:: 1.3
 
 __init__(self, jdf, sql_ctx)
 
 __repr__(self)
 
 agg(self, *exprs)
 Aggregate on the entire :class:`DataFrame` without groups
 (shorthand for ``df.groupBy.agg()``).
 
 >>> df.agg({"age": "max"}).collect()
 [Row(max(age)=5)]
 >>> from pyspark.sql import functions as F
 >>> df.agg(F.min(df.age)).collect()
 [Row(min(age)=2)]
 
 .. versionadded:: 1.3
 
 alias(self, alias)
 Returns a new :class:`DataFrame` with an alias set.
 
 >>> from pyspark.sql.functions import *
 >>> df_as1 = df.alias("df_as1")
 >>> df_as2 = df.alias("df_as2")
 >>> joined_df = df_as1.join(df_as2, col("df_as1.name") == col("df_as2.name"), 'inner')
 >>> joined_df.select(col("df_as1.name"), col("df_as2.name"), col("df_as2.age")).collect()
 [Row(name=u'Alice', name=u'Alice', age=2), Row(name=u'Bob', name=u'Bob', age=5)]
 
 .. versionadded:: 1.3
 
 cache(self)
 Persists with the default storage level (C{MEMORY_ONLY_SER}).
 
 .. versionadded:: 1.3
 
 coalesce(self, numPartitions)
 Returns a new :class:`DataFrame` that has exactly `numPartitions` partitions.
 
 Similar to coalesce defined on an :class:`RDD`, this operation results in a
 narrow dependency, e.g. if you go from 1000 partitions to 100 partitions,
 there will not be a shuffle, instead each of the 100 new partitions will
 claim 10 of the current partitions.
 
 >>> df.coalesce(1).rdd.getNumPartitions()
 1
 
 .. versionadded:: 1.4
 
 collect(self)
 Returns all the records as a list of :class:`Row`.
 
 >>> df.collect()
 [Row(age=2, name=u'Alice'), Row(age=5, name=u'Bob')]
 
 .. versionadded:: 1.3
 
 corr(self, col1, col2, method=None)
 Calculates the correlation of two columns of a DataFrame as a double value.
 Currently only supports the Pearson Correlation Coefficient.
 :func:`DataFrame.corr` and :func:`DataFrameStatFunctions.corr` are aliases of each other.
 
 :param col1: The name of the first column
 :param col2: The name of the second column
 :param method: The correlation method. Currently only supports "pearson"
 
 .. versionadded:: 1.4
 
 count(self)
 Returns the number of rows in this :class:`DataFrame`.
 
 >>> df.count()
 2
 
 .. versionadded:: 1.3
 
 cov(self, col1, col2)
 Calculate the sample covariance for the given columns, specified by their names, as a
 double value. :func:`DataFrame.cov` and :func:`DataFrameStatFunctions.cov` are aliases.
 
 :param col1: The name of the first column
 :param col2: The name of the second column
 
 .. versionadded:: 1.4
 
 crosstab(self, col1, col2)
 Computes a pair-wise frequency table of 

How many partitions will the DataFrame be split into?

In [49]:
dataDF.rdd.getNumPartitions()

Out[ 23 ]: 3

###### A note about DataFrames and queries

When you use DataFrames or Spark SQL, you are building up a _query plan_. Each transformation you apply to a DataFrame adds some information to the query plan. When you finally call an action, which triggers execution of your Spark job, several things happen:

1. Spark's Catalyst optimizer analyzes the query plan (called an _unoptimized logical query plan_) and attempts to optimize it. Optimizations include (but aren't limited to) rearranging and combining `filter()` operations for efficiency, converting `Decimal` operations to more efficient long integer operations, and pushing some operations down into the data source (e.g., a `filter()` operation might be translated to a SQL `WHERE` clause, if the data source is a traditional SQL RDBMS). The result of this optimization phase is an _optimized logical plan_.
2. Once Catalyst has an optimized logical plan, it then constructs multiple _physical_ plans from it. Specifically, it implements the query in terms of lower level Spark RDD operations.
3. Catalyst chooses which physical plan to use via _cost optimization_. That is, it determines which physical plan is the most efficient (or least expensive), and uses that one.
4. Finally, once the physical RDD execution plan is established, Spark actually executes the job.

You can examine the query plan using the `explain()` function on a DataFrame. By default, `explain()` only shows you the final physical plan; however, if you pass it an argument of `True`, it will show you all phases.

(If you want to take a deeper dive into how Catalyst optimizes DataFrame queries, this blog post, while a little old, is an excellent overview: [Deep Dive into Spark SQL's Catalyst Optimizer](https://databricks.com/blog/2015/04/13/deep-dive-into-spark-sqls-catalyst-optimizer.html).)

Let's add a couple transformations to our DataFrame and look at the query plan on the resulting transformed DataFrame. Don't be too concerned if it looks like gibberish. As you gain more experience with Apache Spark, you'll begin to be able to use `explain()` to help you understand more about your DataFrame operations.

In [51]:
newDF = dataDF.distinct().select('*')
newDF.explain(True)

== Parsed Logical Plan ==
'Project [*]
+- Aggregate [last_name#784,first_name#785,ssn#786,occupation#787,age#788L], [last_name#784,first_name#785,ssn#786,occupation#787,age#788L]
 +- LogicalRDD [last_name#784,first_name#785,ssn#786,occupation#787,age#788L], MapPartitionsRDD[396] at applySchemaToPythonRDD at NativeMethodAccessorImpl.java:-2

== Analyzed Logical Plan ==
last_name: string, first_name: string, ssn: string, occupation: string, age: bigint
Project [last_name#784,first_name#785,ssn#786,occupation#787,age#788L]
+- Aggregate [last_name#784,first_name#785,ssn#786,occupation#787,age#788L], [last_name#784,first_name#785,ssn#786,occupation#787,age#788L]
 +- LogicalRDD [last_name#784,first_name#785,ssn#786,occupation#787,age#788L], MapPartitionsRDD[396] at applySchemaToPythonRDD at NativeMethodAccessorImpl.java:-2

== Optimized Logical Plan ==
Aggregate [last_name#784,first_name#785,ssn#786,occupation#787,age#788L], [last_name#784,first_name#785,ssn#786,occupation#787,age#788L]
+- LogicalRDD [last_name#784,first_name#785,ssn#786,occupation#787,age#788L], MapPartitionsRDD[396] at applySchemaToPythonRDD at NativeMethodAccessorImpl.java:-2

== Physical Plan ==
TungstenAggregate(key=[last_name#784,first_name#785,ssn#786,occupation#787,age#788L], functions=[], output=[last_name#784,first_name#785,ssn#786,occupation#787,age#788L])
+- TungstenExchange hashpartitioning(last_name#784,first_name#785,ssn#786,occupation#787,age#788L,200), None
 +- TungstenAggregate(key=[last_name#784,first_name#785,ssn#786,occupation#787,age#788L], functions=[], output=[last_name#784,first_name#785,ssn#786,occupation#787,age#788L])
 +- Scan ExistingRDD[last_name#784,first_name#785,ssn#786,occupation#787,age#788L]

### (3c): Subtract one from each value using _select_

So far, we've created a distributed DataFrame that is split into many partitions, where each partition is stored on a single machine in our cluster.  Let's look at what happens when we do a basic operation on the dataset.  Many useful data analysis operations can be specified as "do something to each item in the dataset".  These data-parallel operations are convenient because each item in the dataset can be processed individually: the operation on one entry doesn't effect the operations on any of the other entries.  Therefore, Spark can parallelize the operation.

One of the most common DataFrame operations is `select()`, and it works more or less like a SQL `SELECT` statement: You can select specific columns from the DataFrame, and you can even use `select()` to create _new_ columns with values that are derived from existing column values. We can use `select()` to create a new column that decrements the value of the existing `age` column.

`select()` is a _transformation_. It returns a new DataFrame that captures both the previous DataFrame and the operation to add to the query (`select`, in this case). But it does *not* actually execute anything on the cluster. When transforming DataFrames, we are building up a _query plan_. That query plan will be optimized, implemented (in terms of RDDs), and executed by Spark _only_ when we call an action.

In [53]:
# Transform dataDF through a select transformation and rename the newly created '(age -1)' column to 'age'
# Because select is a transformation and Spark uses lazy evaluation, no jobs, stages,
# or tasks will be launched when we run this code.
subDF = dataDF.select('last_name', 'first_name', 'ssn', 'occupation', (dataDF.age - 1).alias('age'))

Let's take a look at the query plan.

In [55]:
subDF.explain(True)

== Parsed Logical Plan ==
'Project [unresolvedalias('last_name),unresolvedalias('first_name),unresolvedalias('ssn),unresolvedalias('occupation),(age#788L - 1) AS age#1105]
+- LogicalRDD [last_name#784,first_name#785,ssn#786,occupation#787,age#788L], MapPartitionsRDD[396] at applySchemaToPythonRDD at NativeMethodAccessorImpl.java:-2

== Analyzed Logical Plan ==
last_name: string, first_name: string, ssn: string, occupation: string, age: bigint
Project [last_name#784,first_name#785,ssn#786,occupation#787,(age#788L - cast(1 as bigint)) AS age#1105L]
+- LogicalRDD [last_name#784,first_name#785,ssn#786,occupation#787,age#788L], MapPartitionsRDD[396] at applySchemaToPythonRDD at NativeMethodAccessorImpl.java:-2

== Optimized Logical Plan ==
Project [last_name#784,first_name#785,ssn#786,occupation#787,(age#788L - 1) AS age#1105L]
+- LogicalRDD [last_name#784,first_name#785,ssn#786,occupation#787,age#788L], MapPartitionsRDD[396] at applySchemaToPythonRDD at NativeMethodAccessorImpl.java:-2

== Physical Plan ==
Project [last_name#784,first_name#785,ssn#786,occupation#787,(age#788L - 1) AS age#1105L]
+- Scan ExistingRDD[last_name#784,first_name#785,ssn#786,occupation#787,age#788L]

### (3d) Use _collect_ to view results

<img src="http://spark-mooc.github.io/web-assets/images/cs105x/diagram-3d.png" style="height:700px;float:right"/>

To see a list of elements decremented by one, we need to create a new list on the driver from the the data distributed in the executor nodes.  To do this we can call the `collect()` method on our DataFrame.  `collect()` is often used after transformations to ensure that we are only returning a *small* amount of data to the driver.  This is done because the data returned to the driver must fit into the driver's available memory.  If not, the driver will crash.

The `collect()` method is the first action operation that we have encountered.  Action operations cause Spark to perform the (lazy) transformation operations that are required to compute the values returned by the action.  In our example, this means that tasks will now be launched to perform the `createDataFrame`, `select`, and `collect` operations.

In the diagram, the dataset is broken into four partitions, so four `collect()` tasks are launched. Each task collects the entries in its partition and sends the result to the driver, which creates a list of the values, as shown in the figure below.

Now let's run `collect()` on `subDF`.

In [57]:
# Let's collect the data
results = subDF.collect()
print results

[Row(last_name=u'Harvey', first_name=u'Tracey', ssn=u'160-37-9051', occupation=u'Agricultural engineer', age=38), Row(last_name=u'Green', first_name=u'Leslie', ssn=u'361-94-4342', occupation=u'Teacher, primary school', age=26), Row(last_name=u'Lewis', first_name=u'Tammy', ssn=u'769-27-5887', occupation=u'Scientific laboratory technician', age=20), Row(last_name=u'Cunningham', first_name=u'Kathleen', ssn=u'175-24-7915', occupation=u'Geophysicist/field seismologist', age=41), Row(last_name=u'Marquez', first_name=u'Joshua', ssn=u'310-69-7326', occupation=u'Forensic psychologist', age=25), Row(last_name=u'Summers', first_name=u'Beth', ssn=u'099-90-9730', occupation=u'Best boy', age=42), Row(last_name=u'Jessica', first_name=u'Mrs.', ssn=u'476-06-5497', occupation=u'English as a foreign language teacher', age=42), Row(last_name=u'Turner', first_name=u'Diana', ssn=u'722-09-8354', occupation=u'Psychologist, prison and probation services', age=6), Row(last_name=u'Johnson', first_name=u'Ryan', ssn=u'715-56-1708', occupation=u'Sales executive', age=5), Row(last_name=u'Lewis', first_name=u'Melissa', ssn=u'123-48-8354', occupation=u'Engineer, broadcasting (operations)', age=16), Row(last_name=u'Hernandez', first_name=u'Benjamin', ssn=u'293-22-0265', occupation=u'Scientist, product/process development', age=28), Row(last_name=u'Dixon', first_name=u'Stephanie', ssn=u'041-23-3263', occupation=u'Building control surveyor', age=23), Row(last_name=u'Kim', first_name=u'Leah', ssn=u'725-61-1132', occupation=u'Artist', age=42), Row(last_name=u'Snyder', first_name=u'Leslie', ssn=u'268-79-4330', occupation=u'Chiropractor', age=14), Row(last_name=u'Ortega', first_name=u'Kenneth', ssn=u'077-96-8349', occupation=u'Surveyor, minerals', age=15), Row(last_name=u'Barnes', first_name=u'Ricky', ssn=u'061-88-1648', occupation=u'Production assistant, television', age=32), Row(last_name=u'Adams', first_name=u'Robert', ssn=u'582-28-0099', occupation=u'Manufacturing systems engineer', age=11), Row(last_name=u'Andrade', first_name=u'Sophia', ssn=u'386-07-6013', occupation=u'Social research officer, government', age=5), Row(last_name=u'Weeks', first_name=u'Catherine', ssn=u'363-94-7993', occupation=u'Administrator, education', age=41), Row(last_name=u'Tapia', first_name=u'Thomas', ssn=u'386-39-5490', occupation=u'Horticulturist, commercial', age=10), Row(last_name=u'Arnold', first_name=u'Heather', ssn=u'737-44-0894', occupation=u'Economist', age=7), Row(last_name=u'Bird', first_name=u'Curtis', ssn=u'790-03-8999', occupation=u'Psychotherapist', age=26), Row(last_name=u'Delgado', first_name=u'Megan', ssn=u'789-24-6522', occupation=u'Therapist, drama', age=33), Row(last_name=u'Rojas', first_name=u'Scott', ssn=u'702-94-4924', occupation=u'Press sub', age=39), Row(last_name=u'Love', first_name=u'Gary', ssn=u'117-61-4564', occupation=u'Music tutor', age=14), Row(last_name=u'Gardner', first_name=u'Charles', ssn=u'695-93-4517', occupation=u'Conference centre manager', age=34), Row(last_name=u'Walker', first_name=u'Danielle', ssn=u'164-14-5236', occupation=u'Scientist, water quality', age=41), Row(last_name=u'Dunn', first_name=u'Dominic', ssn=u'452-74-5757', occupation=u'Clinical embryologist', age=13), Row(last_name=u'Scott', first_name=u'Eric', ssn=u'669-41-6318', occupation=u'Surveyor, insurance', age=30), Row(last_name=u'James', first_name=u'Terry', ssn=u'890-73-1947', occupation=u'Hotel manager', age=18), Row(last_name=u'Todd', first_name=u'Joseph', ssn=u'292-28-5631', occupation=u'Biomedical scientist', age=45), Row(last_name=u'Haas', first_name=u'Kimberly', ssn=u'757-69-2225', occupation=u'Engineer, communications', age=17), Row(last_name=u'Vasquez', first_name=u'Lisa', ssn=u'155-76-0838', occupation=u'Estate manager/land agent', age=45), Row(last_name=u'West', first_name=u'Donna', ssn=u'156-67-2394', occupation=u'Sport and exercise psychologist', age=28), Row(last_name=u'Chapman', first_name=u'Chelsea', ssn=u'667-49-0913', occupation=u'Television production assistant

A better way to visualize the data is to use the `show()` method. If you don't tell `show()` how many rows to display, it displays 20 rows.

In [59]:
subDF.show()

+----------+----------+-----------+--------------------+---+
 last_name|first_name| ssn| occupation|age|
+----------+----------+-----------+--------------------+---+
 Harvey| Tracey|160-37-9051|Agricultural engi...| 38|
 Green| Leslie|361-94-4342|Teacher, primary ...| 26|
 Lewis| Tammy|769-27-5887|Scientific labora...| 20|
Cunningham| Kathleen|175-24-7915|Geophysicist/fiel...| 41|
 Marquez| Joshua|310-69-7326|Forensic psycholo...| 25|
 Summers| Beth|099-90-9730| Best boy| 42|
 Jessica| Mrs.|476-06-5497|English as a fore...| 42|
 Turner| Diana|722-09-8354|Psychologist, pri...| 6|
 Johnson| Ryan|715-56-1708| Sales executive| 5|
 Lewis| Melissa|123-48-8354|Engineer, broadca...| 16|
 Hernandez| Benjamin|293-22-0265|Scientist, produc...| 28|
 Dixon| Stephanie|041-23-3263|Building control ...| 23|
 Kim| Leah|725-61-1132| Artist| 42|
 Snyder| Leslie|268-79-4330| Chiropractor| 14|
 Ortega| Kenneth|077-96-8349| Surveyor, minerals| 15|
 Barnes| Ricky|061-88-1648|Production assist...| 32|
 Adams| Robert|582-28-0099|Manufacturing sys...| 11|
 Andrade| Sophia|386-07-6013|Social research o...| 5|
 Weeks| Catherine|363-94-7993|Administrator, ed...| 41|
 Tapia| Thomas|386-39-5490|Horticulturist, c...| 10|
+----------+----------+-----------+--------------------+---+
only showing top 20 rows

If you'd prefer that `show()` not truncate the data, you can tell it not to:

In [61]:
subDF.show(n=30, truncate=False)

+----------+----------+-----------+-------------------------------------------+---+
last_name |first_name|ssn |occupation |age|
+----------+----------+-----------+-------------------------------------------+---+
Harvey |Tracey |160-37-9051|Agricultural engineer |38 |
Green |Leslie |361-94-4342|Teacher, primary school |26 |
Lewis |Tammy |769-27-5887|Scientific laboratory technician |20 |
Cunningham|Kathleen |175-24-7915|Geophysicist/field seismologist |41 |
Marquez |Joshua |310-69-7326|Forensic psychologist |25 |
Summers |Beth |099-90-9730|Best boy |42 |
Jessica |Mrs. |476-06-5497|English as a foreign language teacher |42 |
Turner |Diana |722-09-8354|Psychologist, prison and probation services|6 |
Johnson |Ryan |715-56-1708|Sales executive |5 |
Lewis |Melissa |123-48-8354|Engineer, broadcasting (operations) |16 |
Hernandez |Benjamin |293-22-0265|Scientist, product/process development |28 |
Dixon |Stephanie |041-23-3263|Building control surveyor |23 |
Kim |Leah |725-61-1132|Artist |42 |
Snyder |Leslie |268-79-4330|Chiropractor |14 |
Ortega |Kenneth |077-96-8349|Surveyor, minerals |15 |
Barnes |Ricky |061-88-1648|Production assistant, television |32 |
Adams |Robert |582-28-0099|Manufacturing systems engineer |11 |
Andrade |Sophia |386-07-6013|Social research officer, government |5 |
Weeks |Catherine |363-94-7993|Administrator, education |41 |
Tapia |Thomas |386-39-5490|Horticulturist, commercial |10 |
Arnold |Heather |737-44-0894|Economist |7 |
Bird |Curtis |790-03-8999|Psychotherapist |26 |
Delgado |Megan |789-24-6522|Therapist, drama |33 |
Rojas |Scott |702-94-4924|Press sub |39 |
Love |Gary |117-61-4564|Music tutor |14 |
Gardner |Charles |695-93-4517|Conference centre manager |34 |
Walker |Danielle |164-14-5236|Scientist, water quality |41 |
Dunn |Dominic |452-74-5757|Clinical embryologist |13 |
Scott |Eric |669-41-6318|Surveyor, insurance |30 |
James |Terry |890-73-1947|Hotel manager |18 |
+----------+----------+-----------+-------------------------------------------+---+
only showing top 30 rows

In Databricks, there's an even nicer way to look at the values in a DataFrame: The `display()` helper function.

In [63]:
display(subDF)

last_name,first_name,ssn,occupation,age
Harvey,Tracey,160-37-9051,Agricultural engineer,38
Green,Leslie,361-94-4342,"Teacher, primary school",26
Lewis,Tammy,769-27-5887,Scientific laboratory technician,20
Cunningham,Kathleen,175-24-7915,Geophysicist/field seismologist,41
Marquez,Joshua,310-69-7326,Forensic psychologist,25
Summers,Beth,099-90-9730,Best boy,42
Jessica,Mrs.,476-06-5497,English as a foreign language teacher,42
Turner,Diana,722-09-8354,"Psychologist, prison and probation services",6
Johnson,Ryan,715-56-1708,Sales executive,5
Lewis,Melissa,123-48-8354,"Engineer, broadcasting (operations)",16


### (3e) Use _count_ to get total

One of the most basic jobs that we can run is the `count()` job which will count the number of elements in a DataFrame, using the `count()` action. Since `select()` creates a new DataFrame with the same number of elements as the starting DataFrame, we expect that applying `count()` to each DataFrame will return the same result.

<img src="http://spark-mooc.github.io/web-assets/images/cs105x/diagram-3e.png" style="height:700px;float:right"/>

Note that because `count()` is an action operation, if we had not already performed an action with `collect()`, then Spark would now perform the transformation operations when we executed `count()`.

Each task counts the entries in its partition and sends the result to your SparkContext, which adds up all of the counts. The figure on the right shows what would happen if we ran `count()` on a small example dataset with just four partitions.

In [65]:
print dataDF.count()
print subDF.count()

10000
10000

### (3f) Apply transformation _filter_ and view results with _collect_

Next, we'll create a new DataFrame that only contains the people whose ages are less than 10. To do this, we'll use the `filter()` transformation. (You can also use `where()`, an alias for `filter()`, if you prefer something more SQL-like). The `filter()` method is a transformation operation that creates a new DataFrame from the input DataFrame, keeping only values that match the filter expression.

The figure shows how this might work on the small four-partition dataset.

<img src="http://spark-mooc.github.io/web-assets/images/cs105x/diagram-3f.png" style="height:700px;float:right"/>

To view the filtered list of elements less than 10, we need to create a new list on the driver from the distributed data on the executor nodes.  We use the `collect()` method to return a list that contains all of the elements in this filtered DataFrame to the driver program.

In [67]:
filteredDF = subDF.filter(subDF.age < 10)
filteredDF.show(truncate=False)
filteredDF.count()

+---------+----------+-----------+-------------------------------------------+---+
last_name|first_name|ssn |occupation |age|
+---------+----------+-----------+-------------------------------------------+---+
Turner |Diana |722-09-8354|Psychologist, prison and probation services|6 |
Johnson |Ryan |715-56-1708|Sales executive |5 |
Andrade |Sophia |386-07-6013|Social research officer, government |5 |
Arnold |Heather |737-44-0894|Economist |7 |
Joshua |Mr. |363-83-5358|Hotel manager |8 |
Kelly |Tracy |082-13-6448|Architectural technologist |8 |
Jones |Jackie |412-91-9340|Engineer, drilling |1 |
Church |David |370-59-5122|Museum education officer |6 |
Olson |Melissa |209-27-9609|Surveyor, hydrographic |1 |
Jones |Anne |824-64-4586|Operational researcher |3 |
Harris |Carla |097-46-3461|Higher education careers adviser |5 |
Wyatt |Alicia |188-35-0575|Structural engineer |3 |
Clark |Linda |841-82-4522|Phytotherapist |3 |
Jackson |David |571-22-1560|Sports coach |4 |
Ramirez |Tammy |467-94-4414|Engineer, structural |5 |
Rodriguez|Valerie |662-31-2283|Oceanographer |6 |
Thompson |Cindy |763-02-8605|Engineer, chemical |5 |
Brown |Eric |223-82-8835|Psychologist, clinical |3 |
Woods |Elizabeth |864-11-4296|Herbalist |2 |
Cameron |Danielle |203-14-8171|Exercise physiologist |6 |
+---------+----------+-----------+-------------------------------------------+---+
only showing top 20 rows

 Out[ 32 ]: 2061

(These are some _seriously_ precocious children...)

## Part 4: Python Lambda functions and User Defined Functions

Python supports the use of small one-line anonymous functions that are not bound to a name at runtime.

`lambda` functions, borrowed from LISP, can be used wherever function objects are required. They are syntactically restricted to a single expression. Remember that `lambda` functions are a matter of style and using them is never required - semantically, they are just syntactic sugar for a normal function definition. You can always define a separate normal function instead, but using a `lambda` function is an equivalent and more compact form of coding. Ideally you should consider using `lambda` functions where you want to encapsulate non-reusable code without littering your code with one-line functions.

Here, instead of defining a separate function for the `filter()` transformation, we will use an inline `lambda()` function and we will register that lambda as a Spark _User Defined Function_ (UDF). A UDF is a special wrapper around a function, allowing the function to be used in a DataFrame query.

In [70]:
from pyspark.sql.types import BooleanType
less_ten = udf(lambda s: s < 10, BooleanType())
lambdaDF = subDF.filter(less_ten(subDF.age))
lambdaDF.show()
lambdaDF.count()

+---------+----------+-----------+--------------------+---+
last_name|first_name| ssn| occupation|age|
+---------+----------+-----------+--------------------+---+
 Turner| Diana|722-09-8354|Psychologist, pri...| 6|
 Johnson| Ryan|715-56-1708| Sales executive| 5|
 Andrade| Sophia|386-07-6013|Social research o...| 5|
 Arnold| Heather|737-44-0894| Economist| 7|
 Joshua| Mr.|363-83-5358| Hotel manager| 8|
 Kelly| Tracy|082-13-6448|Architectural tec...| 8|
 Jones| Jackie|412-91-9340| Engineer, drilling| 1|
 Church| David|370-59-5122|Museum education ...| 6|
 Olson| Melissa|209-27-9609|Surveyor, hydrogr...| 1|
 Jones| Anne|824-64-4586|Operational resea...| 3|
 Harris| Carla|097-46-3461|Higher education ...| 5|
 Wyatt| Alicia|188-35-0575| Structural engineer| 3|
 Clark| Linda|841-82-4522| Phytotherapist| 3|
 Jackson| David|571-22-1560| Sports coach| 4|
 Ramirez| Tammy|467-94-4414|Engineer, structural| 5|
Rodriguez| Valerie|662-31-2283| Oceanographer| 6|
 Thompson| Cindy|763-02-8605| Engineer, chemical| 5|
 Brown| Eric|223-82-8835|Psychologist, cli...| 3|
 Woods| Elizabeth|864-11-4296| Herbalist| 2|
 Cameron| Danielle|203-14-8171|Exercise physiolo...| 6|
+---------+----------+-----------+--------------------+---+
only showing top 20 rows

 Out[ 33 ]: 2061

In [71]:
# Let's collect the even values less than 10
even = udf(lambda s: s % 2 == 0, BooleanType())
evenDF = lambdaDF.filter(even(lambdaDF.age))
evenDF.show()
evenDF.count()

+---------+----------+-----------+--------------------+---+
last_name|first_name| ssn| occupation|age|
+---------+----------+-----------+--------------------+---+
 Turner| Diana|722-09-8354|Psychologist, pri...| 6|
 Joshua| Mr.|363-83-5358| Hotel manager| 8|
 Kelly| Tracy|082-13-6448|Architectural tec...| 8|
 Church| David|370-59-5122|Museum education ...| 6|
 Jackson| David|571-22-1560| Sports coach| 4|
Rodriguez| Valerie|662-31-2283| Oceanographer| 6|
 Woods| Elizabeth|864-11-4296| Herbalist| 2|
 Cameron| Danielle|203-14-8171|Exercise physiolo...| 6|
 Clayton| Joanne|027-90-7296|Public house manager| 2|
 Johnson| Scott|216-45-5965| Teacher, music| 6|
 Hayes| Matthew|028-15-3538|Administrator, ed...| 8|
 Rosales| Pamela|391-64-8825| Arts administrator| 2|
 Mendez| Wyatt|804-20-9778|Volunteer coordin...| 8|
 Hill| Robert|899-61-6918|Maintenance engineer| 2|
 Peterson| Amanda|662-80-6804| Mechanical engineer| 4|
 Fry| Charlotte|418-43-3513|Teacher, special ...| 2|
 Morton| Jennifer|250-30-1615| Charity officer| 2|
 Mann| Ashley|463-20-3935| Technical author| 0|
 Rice| Rhonda|274-97-8264|Administrator, ch...| 2|
 Nguyen| Cassandra|044-61-5231|Surveyor, land/ge...| 0|
+---------+----------+-----------+--------------------+---+
only showing top 20 rows

 Out[ 34 ]: 996

## Part 5: Additional DataFrame actions

Let's investigate some additional actions:

* [first()](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.first)
* [take()](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.take)

One useful thing to do when we have a new dataset is to look at the first few entries to obtain a rough idea of what information is available.  In Spark, we can do that using actions like `first()`, `take()`, and `show()`. Note that for the `first()` and `take()` actions, the elements that are returned depend on how the DataFrame is *partitioned*.

Instead of using the `collect()` action, we can use the `take(n)` action to return the first _n_ elements of the DataFrame. The `first()` action returns the first element of a DataFrame, and is equivalent to `take(1)[0]`.

In [73]:
print "first: {0}\n".format(filteredDF.first())

print "Four of them: {0}\n".format(filteredDF.take(4))

first: Row(last_name=u'Turner', first_name=u'Diana', ssn=u'722-09-8354', occupation=u'Psychologist, prison and probation services', age=6)

Four of them: [Row(last_name=u'Turner', first_name=u'Diana', ssn=u'722-09-8354', occupation=u'Psychologist, prison and probation services', age=6), Row(last_name=u'Johnson', first_name=u'Ryan', ssn=u'715-56-1708', occupation=u'Sales executive', age=5), Row(last_name=u'Andrade', first_name=u'Sophia', ssn=u'386-07-6013', occupation=u'Social research officer, government', age=5), Row(last_name=u'Arnold', first_name=u'Heather', ssn=u'737-44-0894', occupation=u'Economist', age=7)]

This looks better:

In [75]:
display(filteredDF.take(4))

last_name,first_name,ssn,occupation,age
Turner,Diana,722-09-8354,"Psychologist, prison and probation services",6
Johnson,Ryan,715-56-1708,Sales executive,5
Andrade,Sophia,386-07-6013,"Social research officer, government",5
Arnold,Heather,737-44-0894,Economist,7


## Part 6: Additional DataFrame transformations

### (6a) _orderBy_

[`orderBy()`](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.distinct) allows you to sort a DataFrame by one or more columns, producing a new DataFrame.

For example, let's get the first five oldest people in the original (unfiltered) DataFrame. We can use the `orderBy()` transformation. `orderBy` takes one or more columns, either as _names_ (strings) or as `Column` objects. To get a `Column` object, we use one of two notations on the DataFrame:

* Pandas-style notation: `filteredDF.age`
* Subscript notation: `filteredDF['age']`

Both of those syntaxes return a `Column`, which has additional methods like `desc()` (for sorting in descending order) or `asc()` (for sorting in ascending order, which is the default).

Here are some examples:

```
dataDF.orderBy(dataDF['age'])  # sort by age in ascending order; returns a new DataFrame
dataDF.orderBy(dataDF.last_name.desc()) # sort by last name in descending order
```

In [78]:
# Get the five oldest people in the list. To do that, sort by age in descending order.
display(dataDF.orderBy(dataDF.age.desc()).take(5))

last_name,first_name,ssn,occupation,age
Smith,Jessica,371-59-8543,Medical physicist,47
Blankenship,Crystal,341-29-9523,Commercial/residential surveyor,47
Meyer,Christine,803-59-5869,Early years teacher,47
George,Wesley,622-72-1540,"Therapist, art",47
Davila,Steven,479-63-8770,Purchasing manager,47


Let's reverse the sort order. Since ascending sort is the default, we can actually use a `Column` object expression or a simple string, in this case. The `desc()` and `asc()` methods are only defined on `Column`. Something like `orderBy('age'.desc())` would not work, because there's no `desc()` method on Python string objects. That's why we needed the column expression. But if we're just using the defaults, we can pass a string column name into `orderBy()`. This is sometimes easier to read.

In [80]:
display(dataDF.orderBy('age').take(5))

last_name,first_name,ssn,occupation,age
Mann,Ashley,463-20-3935,Technical author,1
Nguyen,Cassandra,044-61-5231,"Surveyor, land/geomatics",1
Roach,Samuel,396-43-8078,Financial adviser,1
Collins,Brenda,594-06-8946,Academic librarian,1
Hardin,Benjamin,051-85-9778,"Engineer, drilling",1


### (6b) _distinct_ and _dropDuplicates_

[`distinct()`](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.distinct) filters out duplicate rows, and it considers all columns. Since our data is completely randomly generated (by `fake-factory`), it's extremely unlikely that there are any duplicate rows:

In [82]:
print dataDF.count()
print dataDF.distinct().count()

10000
10000

To demonstrate `distinct()`, let's create a quick throwaway dataset.

In [84]:
tempDF = sqlContext.createDataFrame([("Joe", 1), ("Joe", 1), ("Anna", 15), ("Anna", 12), ("Ravi", 5)], ('name', 'score'))

In [85]:
tempDF.show()

+----+-----+
name|score|
+----+-----+
 Joe| 1|
 Joe| 1|
Anna| 15|
Anna| 12|
Ravi| 5|
+----+-----+

In [86]:
tempDF.distinct().show()

+----+-----+
name|score|
+----+-----+
Ravi| 5|
Anna| 12|
Anna| 15|
 Joe| 1|
+----+-----+

Note that one of the ("Joe", 1) rows was deleted, but both rows with name "Anna" were kept, because all columns in a row must match another row for it to be considered a duplicate.

[`dropDuplicates()`](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.dropDuplicates) is like `distinct()`, except that it allows us to specify the columns to compare. For instance, we can use it to drop all rows where the first name and last name duplicates (ignoring the occupation and age columns).

In [89]:
print dataDF.count()
print dataDF.dropDuplicates(['first_name', 'last_name']).count()

10000
9352

### (6c) _drop_

[`drop()`](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.drop) is like the opposite of `select()`: Instead of selecting specific columns from a DataFrame, it drops a specifed column from a DataFrame.

Here's a simple use case: Suppose you're reading from a 1,000-column CSV file, and you have to get rid of five of the columns. Instead of selecting 995 of the columns, it's easier just to drop the five you don't want.

In [91]:
dataDF.drop('occupation').drop('age').show()

+----------+----------+-----------+
 last_name|first_name| ssn|
+----------+----------+-----------+
 Harvey| Tracey|160-37-9051|
 Green| Leslie|361-94-4342|
 Lewis| Tammy|769-27-5887|
Cunningham| Kathleen|175-24-7915|
 Marquez| Joshua|310-69-7326|
 Summers| Beth|099-90-9730|
 Jessica| Mrs.|476-06-5497|
 Turner| Diana|722-09-8354|
 Johnson| Ryan|715-56-1708|
 Lewis| Melissa|123-48-8354|
 Hernandez| Benjamin|293-22-0265|
 Dixon| Stephanie|041-23-3263|
 Kim| Leah|725-61-1132|
 Snyder| Leslie|268-79-4330|
 Ortega| Kenneth|077-96-8349|
 Barnes| Ricky|061-88-1648|
 Adams| Robert|582-28-0099|
 Andrade| Sophia|386-07-6013|
 Weeks| Catherine|363-94-7993|
 Tapia| Thomas|386-39-5490|
+----------+----------+-----------+
only showing top 20 rows

### (6d) _groupBy_

[`groupBy()`]((http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.groupBy) is one of the most powerful transformations. It allows you to perform aggregations on a DataFrame.

Unlike other DataFrame transformations, `groupBy()` does _not_ return a DataFrame. Instead, it returns a special [GroupedData](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.GroupedData) object that contains various aggregation functions.

The most commonly used aggregation function is [count()](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.GroupedData.count),
but there are others (like [sum()](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.GroupedData.sum), [max()](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.GroupedData.max), and [avg()](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.GroupedData.avg).

These aggregation functions typically create a new column and return a new DataFrame.

In [93]:
dataDF.groupBy('occupation').count().show(truncate=False)

+------------------------------------+-----+
occupation |count|
+------------------------------------+-----+
Agricultural engineer |9 |
Operational researcher |17 |
Textile designer |11 |
Public relations officer |20 |
Politician's assistant |11 |
Personal assistant |11 |
Hotel manager |15 |
Engineer, materials |23 |
Waste management officer |18 |
Counselling psychologist |16 |
Geoscientist |18 |
Therapist, music |19 |
Outdoor activities/education manager|15 |
Physiological scientist |7 |
Housing manager/officer |8 |
Theatre stage manager |16 |
Programmer, systems |14 |
Fitness centre manager |16 |
Fish farm manager |12 |
Musician |16 |
+------------------------------------+-----+
only showing top 20 rows

In [94]:
dataDF.groupBy().avg('age').show(truncate=False)

+--------+
avg(age)|
+--------+
24.447 |
+--------+

We can also use `groupBy()` to do aother useful aggregations:

In [96]:
print "Maximum age: {0}".format(dataDF.groupBy().max('age').first()[0])
print "Minimum age: {0}".format(dataDF.groupBy().min('age').first()[0])

Maximum age: 47
Minimum age: 1

### (6e) _sample_ (optional)

When analyzing data, the [`sample()`](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.sample) transformation is often quite useful. It returns a new DataFrame with a random sample of elements from the dataset.  It takes in a `withReplacement` argument, which specifies whether it is okay to randomly pick the same item multiple times from the parent DataFrame (so when `withReplacement=True`, you can get the same item back multiple times). It takes in a `fraction` parameter, which specifies the fraction elements in the dataset you want to return. (So a `fraction` value of `0.20` returns 20% of the elements in the DataFrame.) It also takes an optional `seed` parameter that allows you to specify a seed value for the random number generator, so that reproducible results can be obtained.

In [98]:
sampledDF = dataDF.sample(withReplacement=False, fraction=0.10)
print sampledDF.count()
sampledDF.show()

987
+---------+-----------+-----------+--------------------+---+
last_name| first_name| ssn| occupation|age|
+---------+-----------+-----------+--------------------+---+
 Summers| Beth|099-90-9730| Best boy| 43|
 Dunn| Dominic|452-74-5757|Clinical embryolo...| 14|
 Scott| Eric|669-41-6318| Surveyor, insurance| 31|
 Vasquez| Lisa|155-76-0838|Estate manager/la...| 46|
 West| Donna|156-67-2394|Sport and exercis...| 29|
 Hill| Steven|194-77-5984|Accountant, chart...| 30|
 Jones| Jackie|412-91-9340| Engineer, drilling| 2|
 Mueller| Amanda|033-74-1619|Administrator, Ci...| 14|
 Bush| Melissa|192-28-9339|Child psychothera...| 39|
 Chapman| Erin|141-37-0804|Radiation protect...| 15|
Henderson| John|310-85-1575| Media buyer| 31|
 Lewis| Victoria|301-47-9178|Scientist, clinic...| 21|
 Anderson| Tanya|670-70-8625| Engineer, chemical| 39|
 Baker| Nancy|477-43-2955|Leisure centre ma...| 31|
 Riley| Melissa|004-37-0436|Newspaper journalist| 17|
 Wright| Nicholas|498-52-6190|International aid...| 12|
 Rice| Rhonda|274-97-8264|Administrator, ch...| 3|
 Young| Brianna|588-41-2984|Learning disabili...| 46|
Armstrong|Christopher|732-92-3213|Research officer,...| 33|
 Shelton| Alex|828-16-3558|Lecturer, further...| 30|
+---------+-----------+-----------+--------------------+---+
only showing top 20 rows

In [99]:
print dataDF.sample(withReplacement=False, fraction=0.05).count()

518

## Part 7: Caching DataFrames and storage options

### (7a) Caching DataFrames

For efficiency Spark keeps your DataFrames in memory. (More formally, it keeps the _RDDs_ that implement your DataFrames in memory.) By keeping the contents in memory, Spark can quickly access the data. However, memory is limited, so if you try to keep too many partitions in memory, Spark will automatically delete partitions from memory to make space for new ones. If you later refer to one of the deleted partitions, Spark will automatically recreate it for you, but that takes time.

So, if you plan to use a DataFrame more than once, then you should tell Spark to cache it. You can use the `cache()` operation to keep the DataFrame in memory. However, you must still trigger an action on the DataFrame, such as `collect()` or `count()` before the caching will occur. In other words, `cache()` is lazy: It merely tells Spark that the DataFrame should be cached _when the data is materialized_. You have to run an action to materialize the data; the DataFrame will be cached as a side effect. The next time you use the DataFrame, Spark will use the cached data, rather than recomputing the DataFrame from the original data.

You can see your cached DataFrame in the "Storage" section of the Spark web UI. If you click on the name value, you can see more information about where the the DataFrame is stored.

In [102]:
# Cache the DataFrame
filteredDF.cache()
# Trigger an action
print filteredDF.count()
# Check if it is cached
print filteredDF.is_cached

2061
True

### (7b) Unpersist and storage options

Spark automatically manages the partitions cached in memory. If it has more partitions than available memory, by default, it will evict older partitions to make room for new ones. For efficiency, once you are finished using cached DataFrame, you can optionally tell Spark to stop caching it in memory by using the DataFrame's `unpersist()` method to inform Spark that you no longer need the cached data.

** Advanced: ** Spark provides many more options for managing how DataFrames cached. For instance, you can tell Spark to spill cached partitions to disk when it runs out of memory, instead of simply throwing old ones away. You can explore the API for DataFrame's [persist()](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrame.persist) operation using Python's [help()](https://docs.python.org/2/library/functions.html?highlight=help#help) command.  The `persist()` operation, optionally, takes a pySpark [StorageLevel](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.StorageLevel) object.

In [104]:
# If we are done with the DataFrame we can unpersist it so that its memory can be reclaimed
filteredDF.unpersist()
# Check if it is cached
print filteredDF.is_cached

False

## ** Part 8: Debugging Spark applications and lazy evaluation **

### How Python is Executed in Spark

Internally, Spark executes using a Java Virtual Machine (JVM). pySpark runs Python code in a JVM using [Py4J](http://py4j.sourceforge.net). Py4J enables Python programs running in a Python interpreter to dynamically access Java objects in a Java Virtual Machine. Methods are called as if the Java objects resided in the Python interpreter and Java collections can be accessed through standard Python collection methods. Py4J also enables Java programs to call back Python objects.

Because pySpark uses Py4J, coding errors often result in a complicated, confusing stack trace that can be difficult to understand. In the following section, we'll explore how to understand stack traces.

### (8a) Challenges with lazy evaluation using transformations and actions

Spark's use of lazy evaluation can make debugging more difficult because code is not always executed immediately. To see an example of how this can happen, let's first define a broken filter function.
Next we perform a `filter()` operation using the broken filtering function.  No error will occur at this point due to Spark's use of lazy evaluation.

The `filter()` method will not be executed *until* an action operation is invoked on the DataFrame.  We will perform an action by using the `count()` method to return a list that contains all of the elements in this DataFrame.

In [108]:
def brokenTen(value):
    """Incorrect implementation of the ten function.

    Note:
        The `if` statement checks an undefined variable `val` instead of `value`.

    Args:
        value (int): A number.

    Returns:
        bool: Whether `value` is less than ten.

    Raises:
        NameError: The function references `val`, which is not available in the local or global
            namespace, so a `NameError` is raised.
    """
    if (val < 10):
        return True
    else:
        return False

btUDF = udf(brokenTen)
brokenDF = subDF.filter(btUDF(subDF.age) == True)

In [109]:
# Now we'll see the error
# Click on the `+` button to expand the error and scroll through the message.
brokenDF.count()

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <ipython-input-53-451d05cfceb7> in <module> () 
 1 # Now we'll see the error 
 2 # Click on the `+` button to expand the error and scroll through the message. 
 ----> 3 brokenDF . count ( ) 

 /databricks/spark/python/pyspark/sql/dataframe.py in count (self) 
 267 2 
 268 """
 --> 269 return int ( self . _jdf . count ( ) ) 
 270 
 271 @ ignore_unicode_prefix 

 /databricks/spark/python/lib/py4j-0.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 811 answer = self . gateway_client . send_command ( command ) 
 812 return_value = get_return_value(
 --> 813 answer, self.gateway_client, self.target_id, self.name)
 814 
 815 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 43 def deco ( * a , ** kw ) : 
 44 try : 
 ---> 45 return f ( * a , ** kw ) 
 46 except py4j . protocol . Py4JJavaError as e : 
 47 s = e . java_exception . toString ( ) 

 /databricks/spark/python/lib/py4j-0.9-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 306 raise Py4JJavaError(
 307 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 308 format(target_id, ".", name), value)
 309 else : 
 310 raise Py4JError(

 Py4JJavaError : An error occurred while calling o612.count.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 69.0 failed 1 times, most recent failure: Lost task 1.0 in stage 69.0 (TID 716, localhost): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
 File "/databricks/spark/python/pyspark/worker.py", line 111, in main
 process()
 File "/databricks/spark/python/pyspark/worker.py", line 106, in process
 serializer.dump_stream(func(split_index, iterator), outfile)
 File "/databricks/spark/python/pyspark/serializers.py", line 263, in dump_stream
 vs = list(itertools.islice(iterator, batch))
 File "/databricks/spark/python/pyspark/sql/functions.py", line 1563, in <lambda>
 func = lambda _, it: map(lambda x: returnType.toInternal(f(*x)), it)
 File "<ipython-input-52-b6d72c054939>", line 17, in brokenTen
NameError: global name 'val' is not defined

	at org.apache.spark.api.python.PythonRunner$$anon$1.read(PythonRDD.scala:166)
	at org.apache.spark.api.python.PythonRunner$$anon$1.<init>(PythonRDD.scala:207)
	at org.apache.spark.api.python.PythonRunner.compute(PythonRDD.scala:125)
	at org.apache.spark.sql.execution.BatchPythonEvaluation$$anonfun$doExecute$1.apply(python.scala:405)
	at org.apache.spark.sql.execution.BatchPythonEvaluation$$anonfun$doExecute$1.apply(python.scala:370)
	at org.apache.spark.rdd.RDD$$anonfun$mapPartitions$1$$anonfun$apply$20.apply(RDD.scala:710)
	at org.apache.spark.rdd.RDD$$anonfun$mapPartitions$1$$anonfun$apply$20.apply(RDD.scala:710)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:38)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:306)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:270)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:38)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:306)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:270)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:38)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:306)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:270)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:38)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:306)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:270)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:38)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:306)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:270)
	at org.apache.spark.scheduler.ShuffleMapTask.runTask(ShuffleMapTask.scala:79)
	at org.apache.spark.scheduler.ShuffleMapTask.runTask(ShuffleM

### (8b) Finding the bug

When the `filter()` method is executed, Spark calls the UDF. Since our UDF has an error in the underlying filtering function `brokenTen()`, an error occurs.

Scroll through the output "Py4JJavaError     Traceback (most recent call last)" part of the cell and first you will see that the line that generated the error is the `count()` method line. There is *nothing wrong with this line*. However, it is an action and that caused other methods to be executed. Continue scrolling through the Traceback and you will see the following error line:

`NameError: global name 'val' is not defined`

Looking at this error line, we can see that we used the wrong variable name in our filtering function `brokenTen()`.

### (8c) Moving toward expert style

As you are learning Spark, I recommend that you write your code in the form:
```
    df2 = df1.transformation1()
    df2.action1()
    df3 = df2.transformation2()
    df3.action2()
```
Using this style will make debugging your code much easier as it makes errors easier to localize - errors in your transformations will occur when the next action is executed.

Once you become more experienced with Spark, you can write your code with the form: `df.transformation1().transformation2().action()`

We can also use `lambda()` functions instead of separately defined functions when their use improves readability and conciseness.

In [112]:
# Cleaner code through lambda use
myUDF = udf(lambda v: v < 10)
subDF.filter(myUDF(subDF.age) == True)

Out[ 54 ]: DataFrame[last_name: string, first_name: string, ssn: string, occupation: string, age: bigint]

### (8d) Readability and code style

To make the expert coding style more readable, enclose the statement in parentheses and put each method, transformation, or action on a separate line.

In [114]:
# Final version
from pyspark.sql.functions import *
(dataDF
 .filter(dataDF.age > 20)
 .select(concat(dataDF.first_name, lit(' '), dataDF.last_name), dataDF.occupation)
 .show(truncate=False)
 )

+------------------------------+--------------------------------------+
concat(first_name, ,last_name)|occupation |
+------------------------------+--------------------------------------+
Tracey Harvey |Agricultural engineer |
Leslie Green |Teacher, primary school |
Tammy Lewis |Scientific laboratory technician |
Kathleen Cunningham |Geophysicist/field seismologist |
Joshua Marquez |Forensic psychologist |
Beth Summers |Best boy |
Mrs. Jessica |English as a foreign language teacher |
Benjamin Hernandez |Scientist, product/process development|
Stephanie Dixon |Building control surveyor |
Leah Kim |Artist |
Ricky Barnes |Production assistant, television |
Catherine Weeks |Administrator, education |
Curtis Bird |Psychotherapist |
Megan Delgado |Therapist, drama |
Scott Rojas |Press sub |
Charles Gardner |Conference centre manager |
Danielle Walker |Scientist, water quality |
Eric Scott |Surveyor, insurance |
Joseph Todd |Biomedical scientist |
Lisa Vasquez |Estate manager/land agent |
+------------------------------+--------------------------------------+
only showing top 20 rows